In [ ]:
!pip install tensorflow_io
!pip install pydub

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.insert(0, '/content/drive/MyDrive/Colab Notebooks/WS2021/ANN')
print("lel")

lel


In [ ]:
import os
import tensorflow as tf

# import globals
# import preprocess_data
# from contrPredCod_model import CPC, InfoNCE
# from classifier_model import Classifier
# from training import main_train_eval_loop

# globals.py

In [ ]:
def initialize():
    global data_generator_arguments
    data_generator_arguments = {}

    global encoder_args
    encoder_args = {}

# hyperparams

In [ ]:
### Hyperparameters ###
# data path
cwd = '/content/drive/MyDrive/Colab Notebooks/WS2021/ANN/final'
cpc_data_path = cwd+"/data/2000songs.zip (Unzipped Files)/2000songs"  # cpc train data
files = os.listdir(cpc_data_path)
filepaths = [os.path.join(cpc_data_path, f) for f in files]
gtzan_feature_path = cwd+'/data/gtzan/features_30_sec.csv'  # baseline features
weight_path = cwd+'/model/cpc'  # where to save weights
load_path = False  # from where to load weights

# CPC data params
# TODO: globals.init if using python scipts and importing
initialize()  # init the global variable
data_generator_arguments = {
    "T": 20,  # timestep
    "k": 10,  # timestep
    "N": 4,  # number
    "full_duration": 5,  # sec
    "original_sr": 22050,  # Hz
    "desired_sr": 4410,  # Hz
    "filepaths": filepaths
    }

# classifier data params
split_rate = 0.8  # train_test split
batch_size_classifier = 16

# encoder params
z_dim = 256  # output dim
encoder_args = {
    "z_dim"          : z_dim,
    "stride_sizes": [5,4,2,2,2],
    "kernel_sizes": [10,8,4,4,4],
    "n_filters":    [512,512,512,512,512],
    "activation":   tf.nn.leaky_relu
}


# AR params
c_dim = 512

# training params
epochs = 2
learning_rate = 1e-5

# Preprocess_data.py

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow.experimental.numpy as tfnp
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# import globals


### FMA data
def decode_audio(audio_path, original_sr, desired_sr, duration):
    """
    Loads and decodes wav file and applies resampling.
    Truncates or pads audio to desired length.

    audio_path: str, path of the audio data to load
    original_sr: float, original sampling rate in Hz
    desired_sr: float, desired sampling rate to which the data should be resampled
    duration: float, desired duration of data in sec
    return: (tf.array, int), 1-d audio file and the desired sampling rate
    """

    audio_binary = tf.io.read_file(audio_path)
    audio, sr = tf.audio.decode_wav(audio_binary,
                                    # mono sound
                                    desired_channels=1,
                                    # crop to get a desired duration
                                    desired_samples=duration * original_sr)
    # if resampling is needed
    if not desired_sr == original_sr:
        audio = tfio.audio.resample(audio, original_sr, desired_sr)
    return tf.squeeze(audio, axis=-1), desired_sr


def gen_batch_CPC():
    """
    Yield a single batch of data by randomly selecting different audios.
    Per default, a single batch is a permutation in which every selected data
    is used as a postive sample once and other as negative samples.

    T: int, #timesteps used for c_t
    k: int, #timesteps to predict
    N: int, number of samples (1 positive + N-1 negative samples)
    original_sr: sampling rate of original audio files
    desired_sr: sampling rate as input to CPC (used for subsampling)
    duration: assumed full duration of an audio file (30s, files get truncated or padded with zeros)
    filepaths: list of all filepaths to all audio files
    """

    # used to get arguments to a generator
    T = data_generator_arguments["T"]
    k = data_generator_arguments["k"]
    N = data_generator_arguments["N"]
    original_sr = data_generator_arguments["original_sr"]
    desired_sr = data_generator_arguments["desired_sr"]
    duration = data_generator_arguments["full_duration"]
    filepaths = data_generator_arguments["filepaths"]
    batch_size = N
    d = int((duration * desired_sr) / (T + k))  # window size (length of a single segment)

    while True:
        # get batch_size random audios with correct resampling and crop
        paths = random.sample(filepaths, batch_size)
        songs = [decode_audio(path, original_sr, desired_sr, duration)[0]
                 for path in paths]

        # get indices to crop negative samples
        max_start_indices = desired_sr * duration - k * d  # max allowed start index for negative samples
        start = tfnp.random.randint(0, max_start_indices, size=(batch_size))  # random indices for negative samples
        end = start + k * d

        # return batch_size samples at the end
        batch = []
        for idx in range(batch_size):
            # selecte a single postive sample
            samples = []  # single postive sample + N-1 negative samples
            positive_sample = songs[idx]
            positive_sample = tf.reshape(positive_sample, (1, T + k, d, 1))
            samples.append(positive_sample)

            # get all negative samples from the same batch with random start point
            [samples.append(tf.reshape(audio[start[i]:end[i]],
                                       (1, k, d, 1))) for i, audio in enumerate(songs) if i != idx]

            # concat to one sample with shape (1, T +k*N, d, 1)
            batch.append(tf.concat(samples, axis=1))

        yield tf.concat(batch, axis=0)  # yield a complete batch from single samples


def get_dataset_CPC():
    """
    Uses a global dictionary "data_generator_arguments" to create a tf dataset from a generator that outputs batches already.

    The data_generator_arguments dictionary has the following arguments:

    T:                  Number of time-steps (each being an audio window) used for prediction
    k:                  Number of time-steps (each being an audio window) to predict
    N:                  Number of negative samples (false/random prediction audio-windows)
    original_sr:        Sampling rate of the audio files
    desired_sr:         Sampling rate used for resampling the audio files (can reduce computational load but cuts high frequencies)
    full_duration:      Length of audio files (shorter files get padded, longer files get cropped)
    filepaths:          List of filepaths to wav files.
    """

    T = data_generator_arguments["T"]
    k = data_generator_arguments["k"]
    N = data_generator_arguments["N"]
    sampling_rate = data_generator_arguments["desired_sr"]
    batch_size = N
    duration = data_generator_arguments["full_duration"]
    sr = data_generator_arguments["desired_sr"]

    # output shape of generator given the arguments
    data_shape = (batch_size, T+k*N, int((duration*sr)/(T+k)), 1)

    train_ds = tf.data.Dataset.from_generator(
        generator = gen_batch_CPC,
        output_signature = tf.TensorSpec(data_shape, 
                                        dtype=tf.dtypes.float32,
                                        name=None)
                                        )
    
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE).cache()

    return train_ds



In [ ]:
### GTZAN data
def create_tfds(inputs, targets, batch_size=None, buffer_size=None, prefetch_factor=None):
    '''
    Create an input pipeline from tf.dataset.
    Adjusted to only take input as there are no labels for autoencoders.
    '''
    dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
    if not buffer_size is None:
        dataset = dataset.shuffle(buffer_size)
    if not batch_size is None:
        dataset = dataset.batch(batch_size)
    if not prefetch_factor is None:
        dataset = dataset.prefetch(prefetch_factor)
    return dataset


def ds_classifier(split_rate, batch_size, path):
    # GTZAN default features
    pd_gtzan = pd.read_csv(path)
    gtzan_feat = pd_gtzan.loc[:, 'chroma_stft_mean':'mfcc20_var'].to_numpy()
    gtzan_labels = np.expand_dims(pd_gtzan.loc[:, 'label'].to_numpy(), axis=1)

    # Standardize the data to make each feature scale invariant
    gtzan_mean = np.expand_dims(np.mean(gtzan_feat, axis=0), axis=0)
    gtzan_std = np.expand_dims(np.std(gtzan_feat, axis=0), axis=0)
    gtzan_feat = (gtzan_feat - gtzan_mean) / gtzan_std

    # one_hot encode labels
    encoder = OneHotEncoder()
    gtzan_targets = encoder.fit_transform(gtzan_labels).toarray()

    # return a tuple of train and test dataset
    split_int = int(split_rate * gtzan_feat.shape[0])
    return (create_tfds(gtzan_feat[:split_int], gtzan_targets[:split_int], batch_size=batch_size,
                        buffer_size=1024, prefetch_factor=tf.data.experimental.AUTOTUNE),
            create_tfds(gtzan_feat[split_int:], gtzan_targets[split_int:], batch_size=batch_size,
                        buffer_size=1024, prefetch_factor=tf.data.experimental.AUTOTUNE))

# encoder_model.py

In [ ]:
import tensorflow as tf


class Encoder (tf.keras.layers.Layer):
    '''
    Encodes an input 1D sequence into an audio window embedding.

    z_dim: size of embedding

    stride_sizes: list of stride arguments for Conv1D layers
    kernel_sizes: list of kernel size arguments for Conv1D layers
    n_filters:    list of filter number arguments for Conv1D layers
    activation:   activation function used in Conv1D layers and for output Dense layer. (e.g. "relu" or tf.nn.relu)
    '''

    def __init__ (self, z_dim, stride_sizes, kernel_sizes, n_filters, activation):
        super(Encoder, self).__init__()

        s = stride_sizes
        k = kernel_sizes
        f = n_filters       

        self.enc_layers = []

        for l in range(len(f)):
            self.enc_layers.append(tf.keras.layers.Conv1D(f[l],k[l],s[l]))
            self.enc_layers.append(tf.keras.layers.BatchNormalization())
            self.enc_layers.append(tf.keras.layers.Activation(activation))
        self.enc_layers.append(tf.keras.layers.Flatten())
        self.enc_layers.append(tf.keras.layers.Dropout(0.1))
        self.enc_layers.append(tf.keras.layers.Dense(z_dim))
        self.enc_layers.append(tf.keras.layers.Activation(activation))
        
    def call (self, x, training):
        # input dim: [batch, T+K*N, window_size, 1]

        for l in self.enc_layers:
            try:
                x = l(x, training)
            except:
                x = l(x)

        # ouput dim:[batch, T+K*N, z]
        return x

# autoreg_model.py

In [ ]:
import tensorflow as tf

# TODO: use transformer
class Autoregressive (tf.keras.layers.Layer):
    '''
    GRU RNN that takes a sequence of audio window embeddings and combines them into a context embedding.
    c_dim: length of context embedding vector
    '''

    def __init__ (self, c_dim):
        super(Autoregressive, self).__init__()
        self.gru = tf.keras.layers.GRU(c_dim, name='ar_context',)
        

    def call (self, z_sequence):
                                    # input dim: [batch, T, z]
        return self.gru(z_sequence) # output dim:[batch, c]


In [ ]:
import numpy as np
import tensorflow as tf

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)  # (1, position, d_model)


def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
            to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                    (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
                                tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
                                tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
                                ])  
 


class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2


class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
        '''
        d_model: z_dim
        num_heads: sets of qkv
        dff: num_units for first ffn layer (d_model is for second layer)
        maximum_position_encoding: T
        '''
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        # TODO: embedding layer isn't needed as the input is already embedded
        # self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding,
                                                self.d_model)

        # TODO: would it possible to have different zdim and cdim?
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                        for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]  # T

        # adding embedding and position encoding.
        # x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        # TODO: we need (batch_size, d_model)
        return x  # (batch_size, input_seq_len, d_model)

# cpc_model.py

In [ ]:
import tensorflow as tf
# from autoreg_model import Autoregressive
# from autoencoder_model import Encoder


def compute_f (z, z_pred):
    '''
    Compute f-scores following eq(3) in the paper to be batch (K x N) matrices.
    Computes similarity (f-)scores as the exp of the dot product of two embeddings. 
    First column of the returned f-score matrix is the postive sample.

    Compute the similarity scores between (k x N) different z_(t+k) and
    k different transformed c_t. First column is the positive sample.
    Currently use a log-bilinear model.
    z: tf.array, k x N different encoded future time segments
    z_pred: tf.array, k different predictions of future time segments
    return: tf.array, score matrix for all combinations of future time segments
    and N different samples.
    '''                                                                                         
                                                                               # z_pred input dim: [batch, K, z]
                                                                               # z input dim:      [batch, K, N, z]
    z = tf.expand_dims(z, axis=-2)                                                  # -> [batch, K, N, 1, z]
    
                                                                        
    pred = tf.repeat(z_pred, repeats=z.shape[2], axis=-2)                           # -> [batch, K*N, z]
    pred = tf.reshape(pred, shape=[z.shape[0],z.shape[1],z.shape[2],z.shape[-1]])   # -> [batch, K, N, z]
    pred = tf.expand_dims(pred, axis=-1)                                            # -> [batch, K, N, z, 1]

    dot_prod = tf.linalg.matmul(z, pred)                                            # -> [batch, K, N, 1, 1]
    # TODO: normalize to avoid inf loss, use cosine sim.?
    dot_prod = dot_prod/ (tf.norm(z)*tf.norm(pred))
    dot_prod = tf.squeeze(dot_prod, axis=[-2,-1])                                   # -> [batch, K, N]
    f_mat = tf.exp(dot_prod)

                                                                                    # output dim: [batch, K, N]
    return f_mat 


# TODO: use GRU?
class Predict_z (tf.keras.layers.Layer):
    '''
    Layer that uses the context embedding c_t to predict K (future) embeddings
    Transform c_t for each timestep t+k to predict future timesteps.
    Currently linear (W_k) for all future timesteps.
    z_dim: int, dimension of encoded future time segments
    K: int, num. future time segments to predict
    '''

    def __init__ (self, z_dim, K):
        super(Predict_z, self).__init__()
        
        # input_dim: [batch, c_dim]
        self.transform_layers = []

        # one linear layer for each future time-step
        for k in tf.range(K):  
            self.transform_layers.append(tf.keras.layers.Dense(z_dim))

    def call(self, c_t):
        
        z_pred = tf.TensorArray(tf.float32, size=len(self.transform_layers))

        for l,layer in enumerate(self.transform_layers):
            # apply linear projection layer for each k
            z_pred = z_pred.write(l, layer(c_t))                        

        z_pred_t = z_pred.stack()
        # output_dim: [batch, K, z]
        return tf.transpose(z_pred_t, perm=[1,0,2])       


class CPC(tf.keras.models.Model):
    '''
    CPC model with encoder, autoregressive model and prediction models.
    num_time_observations: int, num. time segments that is observed
    num_time_future: int, num. time segments to be predicted
    num_samples: int, num. samples to be used, which is 1 + num_neg_samples
    z_dim: int, dimensionality to which inputs are encoded
    c_dim: int, dimensionality of the context vector from AR
    encoder_args: dict, argument dictionary for Encoder model
    return: tf.array, similarity score matrix for (k x N) combinations
    '''


    # TODO: args, encoder and autoregressive
    def __init__(self, num_time_observations, num_time_future, num_samples, z_dim, c_dim, encoder_args):
        super(CPC, self).__init__()
        self.T = num_time_observations
        self.K = num_time_future
        self.N = num_samples
        self.z = z_dim
        self.c = c_dim

        self.g_enc = Encoder(**encoder_args)
        self.g_ar = Autoregressive(self.c)
        self.p_z = Predict_z(z_dim=self.z, K=self.K)

    # @tf.function
    def get_embedding(self, x):
        """
        Calculate embeddings for full input.
        """
        z_t = tf.keras.layers.TimeDistributed(self.g_enc)(x, training=False)
        c_T = self.g_ar(z_t)

        return c_T


    def call(self, x, training=False):
        # input dim: [batch, T+K*N, d, 1]
        # Embedding
        # we need enc first then ar for transformer. Best if we used 'mode' arg.
        z_t = tf.keras.layers.TimeDistributed(  # dim 1 is the temporal dim
            self.g_enc)(x, training=training)  # [batch, T+K*N, z]

        # Split into current observation embeddings and (positive and negative) future embeddings
        z_obs = z_t[:, :self.T]  # [batch, T, z]
        z_future = z_t[:, self.T:]  # [batch, K*N, z]
        z_future = tf.reshape(z_future, [-1, self.K, self.N, self.z])  # [batch, K, N, z]

        # Predict embeddings
        c_T = self.g_ar(z_obs)  # [batch, c]
        z_pred = self.p_z(c_T)  # [batch, K, z]

        # Compute f matrices
        f_mat = compute_f(z_future, z_pred)  # [batch, K, N]
        return f_mat


In [ ]:
class InfoNCE (tf.keras.losses.Loss):
    '''
    Compute InfoNCE loss given a batch of f matrices with dim (K x N)
    '''

    # TODO: trick to avoid inf loss
    def __init__(self):
        self.eps = tf.constant(0.000001)

    def __call__(self, f):
        f = f + self.eps
                                                         # input dim: [batch, K, N]
        denominator = tf.reduce_sum(f, axis=2)           # -> [batch, K]
        losses = - tf.math.log(f[:,:,0] / denominator)  # first column is the positive k predictions
        loss = tf.reduce_mean(losses, axis=None)         # Take mean loss over batch_size and K

        if not tf.math.is_finite(loss):
            tf.print("inf loss")
        return loss

# classifier_model.py

In [ ]:
import tensorflow as tf

class Classifier(tf.keras.Model):
    def __init__(self, classes, act_fct='relu'):
        super(Classifier, self).__init__()
        self.hidden_layers = []

        units = [256, 128, 64, 32]

        for i in range(len(units) - 1):
            self.hidden_layers.append(tf.keras.layers.Dense(units=units[i]))
            self.hidden_layers.append(tf.keras.layers.BatchNormalization())
            self.hidden_layers.append(tf.keras.layers.Activation(act_fct))

        self.output_layer = tf.keras.layers.Dense(units=classes,
                                                  activation='softmax',
                                                  use_bias=False)

    def call(self, x, training=False):
        for l in self.hidden_layers:
            try:
                x = l(x, training)
            except:
                x = l(x)

        return self.output_layer(x)


# training.py

In [ ]:
import tensorflow as tf

@tf.function
def train_step_cpc(model, ds, loss_function, optimizer, steps_per_epoch,
               train_loss_metric=None):

    for batch in ds.take(steps_per_epoch):
        # forward pass with GradientTape
        with tf.GradientTape() as tape:
            prediction = model(batch, training=True)
            loss = loss_function(prediction)

        # backward pass via GradienTape (auto-gradient calc)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # update metrics
        if train_loss_metric is not None:
            train_loss_metric.update_state(loss)
            

@tf.function
def train_step_classify(model, ds, loss_function, optimizer, train_loss_metric, train_acc_metric):
    for x, target in ds:
        with tf.GradientTape() as tape:
            prediction = model(x)
            loss = loss_function(target, prediction)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # update metrics
        train_loss_metric.update_state(loss)
        train_acc_metric.update_state(target, prediction)


@tf.function
def test_step(model, ds, loss_function, loss_metric, acc_metric):
    for x, target in ds:
        prediction = model(x)
        loss = loss_function(target, prediction)

        # update metrics
        loss_metric.update_state(loss)
        acc_metric.update_state(target, prediction)


def eval_metric(metric, val_list):
    result = metric.result()
    metric.reset_states()
    val_list.append(result)
    return result


# TODO: eventually do training in a main loop and not with a single func
def main_train_eval_loop(mode, epochs, model, ds_train, ds_test, loss_function, optimizer, steps_per_epoch,
                         train_loss_metric=None, train_acc_metric=None, test_loss_metric=None, test_acc_metric=None,
                         PATH=False):
    '''
    Call appropriate train and test steps depending on the mode.
    :param mode: str, either 'CPC' or 'classifier'. 'CPC' only calls train, 'classifier' calls train and test steps.
    :param epochs: int, number of epochs to train
    :param model: tf.keras.model, model to train
    :param ds_train: tf.keras.data.dataset, dataset to train on
    :param ds_test: tf.keras.data.dataset, dataset to test on
    :param loss_function: tf.keras.loss, loss function to use
    :param optimizer: tf.keras.optimizer, optimizer to use
    :param steps_per_epoch: int, number of batch to feed per epoch
    :param train_loss_metric: tf.keras.metric
    :param train_acc_metric:
    :param test_loss_metric: tf.keras.metric
    :param test_acc_metric:
    :param PATH: str, path to save to trained model weights
    :return: list of list, 4 metrics for all epochs
    '''

    # arrays to save results for all epochs
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

    for e in range(epochs):
        # Train
        if mode == 'CPC':
            train_step_cpc(model, ds_train, loss_function, optimizer, steps_per_epoch, train_loss_metric)
        elif mode == 'classifier':
            train_step_classify(model, ds_train, loss_function, optimizer, train_loss_metric, train_acc_metric)
        if train_loss_metric is not None:
            eval_metric(train_loss_metric, train_losses)
        if train_acc_metric is not None:
            eval_metric(train_acc_metric, train_accuracies)

        # Evaluate
        if mode == 'classifier':
            test_step(model, ds_test, loss_function, test_loss_metric, test_acc_metric)
        if test_loss_metric is not None:
            eval_metric(test_loss_metric, test_losses)
        if test_acc_metric is not None:
            eval_metric(test_acc_metric, test_accuracies)

    # Save model weights
    if PATH:
        from datetime import datetime
        now = datetime.now()
        save_to = PATH + str(now)[:-10] + ".h5"
        model.save_weights(save_to, overwrite=False)

    return train_losses, train_accuracies, test_losses, test_accuracies

# Main.py

In [ ]:
### (A) Train the CPC model ###
T = data_generator_arguments["T"]
k = data_generator_arguments["k"]
N = data_generator_arguments["N"]
batch_size = N

# Generate a dataset
ds_train_cpc = get_dataset_CPC()


# Define 3 design components
# Model
cpc = CPC(T, k, N, z_dim, c_dim, encoder_args)
# load trained model
if load_path:
    cpc.load_weights(load_path)
# Loss
infonce = InfoNCE()
train_loss_metric_cpc = tf.keras.metrics.Mean('train_loss_CPC')
# Optimizer
adam = tf.keras.optimizers.Adam(learning_rate)


# Training
res_train_loss_cpc, *_ = main_train_eval_loop('CPC', epochs, cpc, ds_train_cpc, None, infonce, adam, steps_per_epoch=batch_size,
                                          train_loss_metric=train_loss_metric_cpc, PATH=weight_path)
# TODO: visualize loss-epoch using tensorboard
######



### (B,C) Get a baseline performance of a classifier using GTZAN pre-found features ###
# Generate a dataset
gtzan_train, gtzan_test = ds_classifier(split_rate, batch_size, gtzan_feature_path)


# Define 3 design components
# Model
num_classes = list(gtzan_train.take(1).as_numpy_iterator())[0][1].shape[1]
classi1 = Classifier(num_classes)
# Loss
cce = tf.keras.losses.CategoricalCrossentropy()
train_acc_metric_classi1 = tf.keras.metrics.CategoricalAccuracy('train_accuracy_classi1')
train_loss_metric_classi1 = tf.keras.metrics.Mean('train_loss_classi1')
test_acc_metric_classi1 = tf.keras.metrics.CategoricalAccuracy('test_accuracy_classi1')
test_loss_metric_classi1 = tf.keras.metrics.Mean('test_loss_classi1')

 
# Training and testing
res_train_loss_c1, res_train_acc_c1, res_test_loss_c1, res_test_acc_c1 = main_train_eval_loop(
    'classifier', epochs, classi1, gtzan_train, gtzan_test, cce, adam, steps_per_epoch=8,
    train_loss_metric=train_loss_metric_classi1, train_acc_metric=train_acc_metric_classi1,
     test_loss_metric=test_loss_metric_classi1, test_acc_metric=test_acc_metric_classi1)


In [ ]:
### (D,E) Classify with learned features from CPC model
# Get features using trained CPC
import tensorflow_datasets as tfds
gtzan_ds = tfds.load('gtzan', split='train', as_supervised=True).shuffle(1024).prefetch(tf.data.experimental.AUTOTUNE)
gtzan_ds_train = gtzan_ds.take(int(1000*split_rate))
gtzan_ds_test = gtzan_ds.skip(int(1000*split_rate))


def get_embedded_ds(ds, model):
    features = []
    labels = []
    for audio, label in ds:
        audio = audio[:(T+k)*d]  # if GTZAN happends to be longer
        audio = tf.reshape(audio, (1, T+k, d, 1))  # 
        print(audio.shape)
        # TODO: Value for attr 'T' of int64 is not in the list of allowed values:
        features.append(model.get_embedding(audio))
        labels.append(label)
    print("features, labels")
    print(np.array(features).shape, np.array(labels).shape)
    return tf.data.Dataset.from_tensor_slices((features,labels))


cpc_train_features = get_embedded_ds(gtzan_ds_train, cpc).batch(batch_size)
cpc_test_features = get_embedded_ds(gtzan_ds_test, cpc).batch(batch_size)

# Design components
# model
classi2 = Classifier(num_classes)
# metrics
train_acc_metric_classi2 = tf.keras.metrics.CategoricalAccuracy('train_accuracy_classi2')
train_loss_metric_classi2 = tf.keras.metrics.Mean('train_loss_classi2')
test_acc_metric_classi2 = tf.keras.metrics.CategoricalAccuracy('test_accuracy_classi2')
test_loss_metric_classi2 = tf.keras.metrics.Mean('test_loss_classi2')

# Training and testing
res_train_loss_c2, res_train_acc_c2, res_test_loss_c2, res_test_acc_c2 = main_train_eval_loop(
    'classifier', epochs, classi2, cpc_train_features, cpc_test_features, cce, adam, steps_per_epoch=8,
    train_loss_metric=train_loss_metric_classi2, train_acc_metric=train_acc_metric_classi2,
     test_loss_metric=test_loss_metric_classi2, test_acc_metric=test_acc_metric_classi2)

Shuffling and writing examples to /root/tensorflow_datasets/gtzan/1.0.0.incomplete5BWH2D/gtzan-train.tfrecord


Dataset gtzan downloaded and prepared to /root/tensorflow_datasets/gtzan/1.0.0. Subsequent calls will reuse this data.


NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
for i,l in gtzan_ds.take(1):
    print(i,l)

In [ ]:
# TODO: automatic result and figure saving
import matplotlib.pyplot as plt
fig, ax = plt.subplots(ncols=2)

ax[0].plot(np.array(res_train_loss_c1), color='r')
ax[0].plot(np.array(res_test_loss_c1), color='b')
ax[0].set(title='cce loss')
ax[1].plot(np.array(res_train_acc_c1), color='r')
ax[1].plot(np.array(res_test_acc_c1), color='b')
ax[1].set(title='accuracy')